# 주식 정보 데이터 수집
앞서 주식 정보를 가져왔었다. 하지만 해당 날짜 데이터만 가져왔을뿐 그전 데이터에 대해서는 없다. 과거데이터를 얻고 싶다면 종목코드만 알고 있으면 언제든지 정보를 가져 올수 있다. 그래서 미리 데이터 수집을 더 좋게 하기 위해 사전 연습을 해보도록 하였다.

# 1. 라이브러리 및 데이터 불러오기

In [1]:
import pandas as pd
import time
from pykrx import stock

이전 [KRX(한국거래소)주식 정보 가져오기](https://uikang.tistory.com/86) 블로깅에서 저장한 csv파일을 가져오도록 하자 

In [2]:
df = pd.read_csv('stock_list.csv')
df

,종목코드,종목명,시장구분
0,060310,3S,KOSDAQ
1,095570,AJ네트웍스,KOSPI
2,006840,AK홀딩스,KOSPI
3,054620,APS홀딩스,KOSDAQ
4,265520,AP시스템,KOSDAQ
...,...,...,...
2599,000547,흥국화재2우B,KOSPI
2600,000545,흥국화재우,KOSPI
2601,003280,흥아해운,KOSPI
2602,037440,희림,KOSDAQ


# 2. 범위 설정
범위 설정은 2021/11/27 ~ 2021/12/01 KOSPI데이터만을 사용해 데이터를 만들어보자.

In [3]:
start_day = "20211127"
end_day = "20211201"

In [4]:
kospi_df = df[df["시장구분"] == "KOSPI"] # KOSPI만 가져오기

In [5]:
kospi_df = kospi_df.head(10) # 10개의 회사 데이터만 사용. 
kospi_df

,종목코드,종목명,시장구분
1,095570,AJ네트웍스,KOSPI
2,006840,AK홀딩스,KOSPI
6,027410,BGF,KOSPI
7,282330,BGF리테일,KOSPI
9,138930,BNK금융지주,KOSPI
10,001460,BYC,KOSPI
11,001465,BYC우,KOSPI
13,001040,CJ,KOSPI
14,079160,CJ CGV,KOSPI
16,00104K,CJ4우(전환),KOSPI


# 3. 인덱스 설정

In [6]:
kospi_df = kospi_df.reset_index(drop = True) # reset_index 인덱스를 재배열하는 함수, drop = True 기존 인덱스를 제거후 재배열
kospi_df

,종목코드,종목명,시장구분
0,095570,AJ네트웍스,KOSPI
1,006840,AK홀딩스,KOSPI
2,027410,BGF,KOSPI
3,282330,BGF리테일,KOSPI
4,138930,BNK금융지주,KOSPI
5,001460,BYC,KOSPI
6,001465,BYC우,KOSPI
7,001040,CJ,KOSPI
8,079160,CJ CGV,KOSPI
9,00104K,CJ4우(전환),KOSPI


# 4. for 문 이용 주식 데이터 불러오기

In [7]:
stock_df_total = pd.DataFrame()
for i in range(len(kospi_df)):
    stock_code = kospi_df['종목코드'][i]
    stock_name = kospi_df['종목명'][i]
    market = kospi_df['시장구분'][i]
    
 #stock.get_market_ohlcv_by_date 주식데이터 시작일,종료일,티커 파라미터를 넣어주면 일자별로 정렬하여 DataFrame으로 반환합니다
    stock_df = stock.get_market_ohlcv_by_date(fromdate=start_day, todate=end_day, ticker=stock_code) 
    # 위 데이터를 가져오면 날짜,시가,고가,저가,..등만 나오므로 칼럼을 추가해주도록 하자.
    stock_df["종목명"] = stock_name 
    stock_df["종목코드"] = stock_code
    stock_df["시장구분"] = market
    
    
    time.sleep(1) # time.sleep(1) 1초마다 데이터 실행 (데이터가 많으면 한꺼번에 처리하려해 서버 팅길수 있으므로 설정 )
    print(str(i) + " 번째 "+ stock_name) # 잘 실행되는지 확인하기위해 설정 해주었다.
    
    # pd.concat 데이터 프레임 이어 붙히기
    stock_df_total = pd.concat([stock_df_total,stock_df])

0 번째 AJ네트웍스
1 번째 AK홀딩스
2 번째 BGF
3 번째 BGF리테일
4 번째 BNK금융지주
5 번째 BYC
6 번째 BYC우
7 번째 CJ
8 번째 CJ CGV
9 번째 CJ4우(전환)


In [9]:
stock_df_total

,시가,고가,저가,종가,거래량,종목명,종목코드,시장구분
날짜,,,,,,,,
2021-11-29,5600,5750,5480,5510,181622,AJ네트웍스,095570,KOSPI
2021-11-30,5580,5630,5190,5350,141270,AJ네트웍스,095570,KOSPI
2021-12-01,5300,5380,5170,5380,72085,AJ네트웍스,095570,KOSPI
2021-11-29,19500,20300,19150,20100,51924,AK홀딩스,006840,KOSPI
2021-11-30,19900,20300,18700,18950,44817,AK홀딩스,006840,KOSPI
2021-12-01,18950,19300,18550,19050,57400,AK홀딩스,006840,KOSPI
2021-11-29,5310,5400,5190,5220,490612,BGF,027410,KOSPI
2021-11-30,5250,5290,4910,4910,793483,BGF,027410,KOSPI
2021-12-01,4870,4975,4820,4940,256977,BGF,027410,KOSPI


이렇게 3일치 주식데이터를 10개의 회사별로 수집하였다. 더많은 데이터를 수집하려면 용량이 엄청 커지므로 DB로 해야 하니 좀더 배운뒤 DB로 저장 후 주식데이터를 수집하는 방법을 한번 정리해서 올려보도록 하겠다.